In [4]:
# ─────────────────────────────────────────────────────────────────────────────
# NEO4J KNOWLEDGE GRAPH AGENT
# Multi-Agent Viral Content Pipeline — Step 3
#
# Reads:  scout_filtered.json  (25 top posts from Step 1c)
# Writes: neo4j_upsert_log.json
#
# Graph schema:
#   (:Topic)         → AI topic (OpenAI agents, Claude AI, etc.)
#   (:Platform)      → twitter, youtube, reddit, etc.
#   (:Post)          → individual content piece
#   (:Signal)        → viral signal word
#   (:Trend)         → aggregated trend per topic
#
# Relationships:
#   (Post)-[:ABOUT]->(Topic)
#   (Post)-[:POSTED_ON]->(Platform)
#   (Post)-[:HAS_SIGNAL]->(Signal)
#   (Topic)-[:HAS_TREND]->(Trend)
#
# !pip install neo4j -q
# ─────────────────────────────────────────────────────────────────────────────

import json
import time
from getpass import getpass
from datetime import datetime, timezone
from collections import defaultdict, Counter
from dataclasses import dataclass, field, asdict
from typing import List, Optional

from neo4j import GraphDatabase

# ─────────────────────────────────────────────────────────────────────────────
# CONFIG
# ─────────────────────────────────────────────────────────────────────────────

INPUT_FILE   = "scout_filtered.json"   # output from Step 1c
OUTPUT_FILE  = "neo4j_upsert_log.json"

# Neo4j connection — fill in your AuraDB or local instance
NEO4J_URI      = "neo4j+s://YOUR_AURA_INSTANCE.databases.neo4j.io"  # AuraDB free tier URI
NEO4J_USER     = "neo4j"
NEO4J_PASSWORD = ""   # will be prompted securely

# Pipeline run label — lets you track multiple runs in the graph
PIPELINE_RUN_ID = f"run_{datetime.now(timezone.utc).strftime('%Y%m%d_%H%M%S')}"

# ─────────────────────────────────────────────────────────────────────────────
# DATA STRUCTURES
# ─────────────────────────────────────────────────────────────────────────────

@dataclass
class UpsertResult:
    node_type:    str
    operation:    str   # "created" | "merged"
    node_id:      str
    properties:   dict  = field(default_factory=dict)

@dataclass
class Neo4jLog:
    agent:          str  = "neo4j_agent"
    pipeline_run:   str  = ""
    timestamp:      str  = field(default_factory=lambda: datetime.now(timezone.utc).isoformat())
    posts_ingested: int  = 0
    nodes_created:  int  = 0
    nodes_merged:   int  = 0
    rels_created:   int  = 0
    upserts:        List[UpsertResult] = field(default_factory=list)
    trend_summary:  dict = field(default_factory=dict)
    status:         str  = "success"
    error:          Optional[str] = None

# ─────────────────────────────────────────────────────────────────────────────
# CYPHER QUERIES
# ─────────────────────────────────────────────────────────────────────────────

# Constraints — run once to ensure uniqueness
CONSTRAINTS = [
    "CREATE CONSTRAINT topic_name IF NOT EXISTS FOR (t:Topic) REQUIRE t.name IS UNIQUE",
    "CREATE CONSTRAINT platform_name IF NOT EXISTS FOR (p:Platform) REQUIRE p.name IS UNIQUE",
    "CREATE CONSTRAINT post_url IF NOT EXISTS FOR (p:Post) REQUIRE p.url IS UNIQUE",
    "CREATE CONSTRAINT signal_word IF NOT EXISTS FOR (s:Signal) REQUIRE s.word IS UNIQUE",
    "CREATE CONSTRAINT trend_id IF NOT EXISTS FOR (t:Trend) REQUIRE t.trend_id IS UNIQUE",
]

# Upsert a Topic node
MERGE_TOPIC = """
MERGE (t:Topic {name: $name})
ON CREATE SET
    t.created_at    = $now,
    t.post_count    = 1,
    t.total_views   = $views,
    t.total_likes   = $likes,
    t.avg_score     = $score,
    t.pipeline_run  = $run_id
ON MATCH SET
    t.post_count    = t.post_count + 1,
    t.total_views   = t.total_views + $views,
    t.total_likes   = t.total_likes + $likes,
    t.avg_score     = (t.avg_score + $score) / 2.0,
    t.last_updated  = $now
RETURN t
"""

# Upsert a Platform node
MERGE_PLATFORM = """
MERGE (p:Platform {name: $name})
ON CREATE SET
    p.created_at   = $now,
    p.post_count   = 1,
    p.pipeline_run = $run_id
ON MATCH SET
    p.post_count   = p.post_count + 1,
    p.last_updated = $now
RETURN p
"""

# Upsert a Post node
MERGE_POST = """
MERGE (p:Post {url: $url})
ON CREATE SET
    p.title          = $title,
    p.platform       = $platform,
    p.topic          = $topic,
    p.content_type   = $content_type,
    p.final_score    = $final_score,
    p.relevance_score= $relevance_score,
    p.views          = $views,
    p.likes          = $likes,
    p.published_date = $published_date,
    p.viral_signals  = $viral_signals,
    p.snippet        = $snippet,
    p.pipeline_run   = $run_id,
    p.created_at     = $now
ON MATCH SET
    p.final_score    = $final_score,
    p.views          = $views,
    p.likes          = $likes,
    p.last_updated   = $now
RETURN p
"""

# Upsert a Signal node
MERGE_SIGNAL = """
MERGE (s:Signal {word: $word})
ON CREATE SET
    s.count        = 1,
    s.created_at   = $now,
    s.pipeline_run = $run_id
ON MATCH SET
    s.count        = s.count + 1,
    s.last_seen    = $now
RETURN s
"""

# Relationships
REL_POST_ABOUT_TOPIC = """
MATCH (p:Post {url: $url})
MATCH (t:Topic {name: $topic})
MERGE (p)-[r:ABOUT]->(t)
ON CREATE SET r.created_at = $now
RETURN r
"""

REL_POST_ON_PLATFORM = """
MATCH (p:Post {url: $url})
MATCH (pl:Platform {name: $platform})
MERGE (p)-[r:POSTED_ON]->(pl)
ON CREATE SET r.created_at = $now
RETURN r
"""

REL_POST_HAS_SIGNAL = """
MATCH (p:Post {url: $url})
MATCH (s:Signal {word: $word})
MERGE (p)-[r:HAS_SIGNAL]->(s)
ON CREATE SET r.created_at = $now
RETURN r
"""

# Upsert a Trend node (one per topic per pipeline run)
MERGE_TREND = """
MERGE (tr:Trend {trend_id: $trend_id})
ON CREATE SET
    tr.topic           = $topic,
    tr.pipeline_run    = $run_id,
    tr.post_count      = $post_count,
    tr.top_platform    = $top_platform,
    tr.top_signal      = $top_signal,
    tr.avg_score       = $avg_score,
    tr.total_views     = $total_views,
    tr.total_likes     = $total_likes,
    tr.top_post_title  = $top_post_title,
    tr.top_post_url    = $top_post_url,
    tr.created_at      = $now
ON MATCH SET
    tr.post_count      = $post_count,
    tr.avg_score       = $avg_score,
    tr.total_views     = $total_views,
    tr.last_updated    = $now
RETURN tr
"""

REL_TOPIC_HAS_TREND = """
MATCH (t:Topic {name: $topic})
MATCH (tr:Trend {trend_id: $trend_id})
MERGE (t)-[r:HAS_TREND]->(tr)
ON CREATE SET r.created_at = $now
RETURN r
"""

# ─────────────────────────────────────────────────────────────────────────────
# NEO4J AGENT
# ─────────────────────────────────────────────────────────────────────────────

class Neo4jAgent:

    def __init__(self, uri: str, user: str, password: str, run_id: str):
        self.driver  = GraphDatabase.driver(uri, auth=(user, password))
        self.run_id  = run_id
        self.log     = Neo4jLog(pipeline_run=run_id)
        print(f"✅ Connected to Neo4j — run: {run_id}")

    def close(self):
        self.driver.close()

    def _run(self, query: str, params: dict) -> list:
        with self.driver.session() as session:
            result = session.run(query, **params)
            return list(result)

    # ── Setup ──────────────────────────────────────────────────────────────

    def setup_constraints(self):
        print("\n⚙️  Setting up constraints...")
        for c in CONSTRAINTS:
            try:
                self._run(c, {})
                print(f"   ✓ {c[:60]}...")
            except Exception as e:
                print(f"   ⚠️  {e}")

    # ── Upsert helpers ─────────────────────────────────────────────────────

    def _now(self) -> str:
        return datetime.now(timezone.utc).isoformat()

    def _upsert_topic(self, topic: str, views: int, likes: int, score: float):
        self._run(MERGE_TOPIC, {
            "name": topic, "now": self._now(),
            "views": views, "likes": likes,
            "score": score, "run_id": self.run_id
        })

    def _upsert_platform(self, platform: str):
        self._run(MERGE_PLATFORM, {
            "name": platform, "now": self._now(), "run_id": self.run_id
        })

    def _upsert_post(self, post: dict):
        eng = post.get("engagement_metrics", {})
        self._run(MERGE_POST, {
            "url":            post["url"],
            "title":          post["title"][:200],
            "platform":       post["platform"],
            "topic":          post["topic"],
            "content_type":   post["content_type"],
            "final_score":    post.get("final_score", 0.0),
            "relevance_score":post.get("relevance_score", 0.0),
            "views":          int(eng.get("views", 0)),
            "likes":          int(eng.get("likes", 0)),
            "published_date": post.get("published_date") or "",
            "viral_signals":  post.get("viral_signals", []),
            "snippet":        post.get("snippet", "")[:500],
            "run_id":         self.run_id,
            "now":            self._now(),
        })

    def _upsert_signal(self, word: str):
        self._run(MERGE_SIGNAL, {
            "word": word, "now": self._now(), "run_id": self.run_id
        })

    def _upsert_relationships(self, post: dict):
        params_base = {"url": post["url"], "now": self._now()}

        self._run(REL_POST_ABOUT_TOPIC, {**params_base, "topic": post["topic"]})
        self._run(REL_POST_ON_PLATFORM, {**params_base, "platform": post["platform"]})

        for signal in post.get("viral_signals", []):
            self._run(REL_POST_HAS_SIGNAL, {**params_base, "word": signal})

    def _upsert_trend(self, topic: str, posts: list):
        if not posts:
            return

        views_total = sum(int(p.get("engagement_metrics", {}).get("views", 0)) for p in posts)
        likes_total = sum(int(p.get("engagement_metrics", {}).get("likes", 0)) for p in posts)
        avg_score   = sum(p.get("final_score", 0) for p in posts) / len(posts)

        # Top platform for this topic
        plat_counts = Counter(p["platform"] for p in posts)
        top_platform = plat_counts.most_common(1)[0][0]

        # Top signal for this topic
        all_signals = [s for p in posts for s in p.get("viral_signals", [])]
        top_signal  = Counter(all_signals).most_common(1)[0][0] if all_signals else "none"

        # Best post
        best_post = max(posts, key=lambda p: p.get("final_score", 0))

        trend_id = f"{topic.replace(' ', '_')}_{self.run_id}"

        self._run(MERGE_TREND, {
            "trend_id":      trend_id,
            "topic":         topic,
            "run_id":        self.run_id,
            "post_count":    len(posts),
            "top_platform":  top_platform,
            "top_signal":    top_signal,
            "avg_score":     round(avg_score, 4),
            "total_views":   views_total,
            "total_likes":   likes_total,
            "top_post_title":best_post["title"][:200],
            "top_post_url":  best_post["url"],
            "now":           self._now(),
        })
        self._run(REL_TOPIC_HAS_TREND, {
            "topic":    topic,
            "trend_id": trend_id,
            "now":      self._now(),
        })

        return {
            "topic":        topic,
            "post_count":   len(posts),
            "total_views":  views_total,
            "total_likes":  likes_total,
            "avg_score":    round(avg_score, 4),
            "top_platform": top_platform,
            "top_signal":   top_signal,
            "top_post":     best_post["title"][:80],
        }

    # ── Main ingest ────────────────────────────────────────────────────────

    def ingest(self, posts: list) -> Neo4jLog:
        print(f"\n{'='*60}")
        print(f"🕸️  NEO4J AGENT — ingesting {len(posts)} posts")
        print(f"{'='*60}")

        self.setup_constraints()

        # Group by topic for trend aggregation
        by_topic = defaultdict(list)

        print(f"\n📥 Upserting nodes + relationships...")
        for i, post in enumerate(posts):
            topic    = post.get("topic", "unknown")
            platform = post.get("platform", "unknown")
            eng      = post.get("engagement_metrics", {})
            views    = int(eng.get("views", 0))
            likes    = int(eng.get("likes", 0))
            score    = post.get("final_score", 0.0)

            print(f"   [{i+1:02d}/{len(posts)}] {platform:10s} | {topic:25s} | {post['title'][:45]}")

            # Upsert nodes
            self._upsert_topic(topic, views, likes, score)
            self._upsert_platform(platform)
            self._upsert_post(post)
            for signal in post.get("viral_signals", []):
                self._upsert_signal(signal)

            # Upsert relationships
            self._upsert_relationships(post)

            by_topic[topic].append(post)
            self.log.posts_ingested += 1
            time.sleep(0.05)   # gentle rate limiting

        # Upsert trend nodes per topic
        print(f"\n📊 Computing + upserting Trend nodes...")
        trend_summary = {}
        for topic, topic_posts in by_topic.items():
            trend = self._upsert_trend(topic, topic_posts)
            if trend:
                trend_summary[topic] = trend
                print(f"   ✓ {topic:30s} | {trend['post_count']} posts | "
                      f"{trend['total_views']:,} views | top: {trend['top_platform']}")

        self.log.trend_summary = trend_summary
        return self.log

# ─────────────────────────────────────────────────────────────────────────────
# DISPLAY
# ─────────────────────────────────────────────────────────────────────────────

def print_graph_summary(log: Neo4jLog, posts: list):
    print(f"\n{'='*60}")
    print(f"🕸️  KNOWLEDGE GRAPH SUMMARY")
    print(f"{'='*60}")
    print(f"   Run ID         : {log.pipeline_run}")
    print(f"   Posts ingested : {log.posts_ingested}")
    print(f"   Status         : {log.status}")

    print(f"\n   Topics in graph:")
    for topic, trend in log.trend_summary.items():
        print(f"   ┌─ 🏷️  {topic}")
        print(f"   │   posts     : {trend['post_count']}")
        print(f"   │   views     : {trend['total_views']:,}")
        print(f"   │   likes     : {trend['total_likes']:,}")
        print(f"   │   avg score : {trend['avg_score']}")
        print(f"   │   top plat  : {trend['top_platform']}")
        print(f"   │   top signal: {trend['top_signal']}")
        print(f"   └─ 🏆 {trend['top_post'][:60]}")
        print()

    # Platform distribution
    plat_counts = Counter(p["platform"] for p in posts)
    print(f"   Platform distribution:")
    for plat, cnt in plat_counts.most_common():
        bar = "█" * cnt
        print(f"     {plat:12s} {bar} {cnt}")

    # Top signals across all posts
    all_signals = [s for p in posts for s in p.get("viral_signals", [])]
    sig_counts  = Counter(all_signals)
    print(f"\n   Top viral signals in graph:")
    for sig, cnt in sig_counts.most_common(8):
        print(f"     {sig:25s}: {cnt}")

    print(f"\n{'='*60}")

def print_cypher_queries(log: Neo4jLog):
    """Print useful Cypher queries to explore the graph."""
    print(f"\n{'='*60}")
    print(f"🔍 EXPLORE YOUR GRAPH — copy these into Neo4j Browser")
    print(f"{'='*60}")

    queries = [
        ("All topics + post counts",
         "MATCH (t:Topic)<-[:ABOUT]-(p:Post) RETURN t.name, count(p) as posts, t.total_views as views ORDER BY views DESC"),

        ("Top posts by score",
         "MATCH (p:Post) RETURN p.title, p.platform, p.final_score, p.views ORDER BY p.final_score DESC LIMIT 10"),

        ("Trending signals",
         "MATCH (s:Signal)<-[:HAS_SIGNAL]-(p:Post) RETURN s.word, count(p) as post_count ORDER BY post_count DESC LIMIT 10"),

        ("Topic → Platform heatmap",
         "MATCH (p:Post)-[:ABOUT]->(t:Topic), (p)-[:POSTED_ON]->(pl:Platform) RETURN t.name, pl.name, count(p) as posts ORDER BY posts DESC"),

        ("Full graph (small)",
         "MATCH (p:Post)-[:ABOUT]->(t:Topic) RETURN p, t LIMIT 50"),

        ("Trends this run",
         f"MATCH (t:Topic)-[:HAS_TREND]->(tr:Trend {{pipeline_run: '{log.pipeline_run}'}}) RETURN t.name, tr.post_count, tr.total_views, tr.top_signal ORDER BY tr.total_views DESC"),

        ("Posts with most viral signals",
         "MATCH (p:Post)-[:HAS_SIGNAL]->(s:Signal) WITH p, count(s) as signal_count RETURN p.title, p.platform, signal_count ORDER BY signal_count DESC LIMIT 10"),
    ]

    for title, cypher in queries:
        print(f"\n   // {title}")
        print(f"   {cypher}")

    print(f"\n{'='*60}")

# ─────────────────────────────────────────────────────────────────────────────
# MAIN
# ─────────────────────────────────────────────────────────────────────────────

if __name__ == "__main__":

    # ── Load filtered posts from Step 1c ─────────────────────
    print(f"📂 Loading {INPUT_FILE}...")
    with open(INPUT_FILE, "r") as f:
        filtered_data = json.load(f)

    posts = filtered_data.get("posts", [])
    print(f"   {len(posts)} posts loaded")
    print(f"   Source run: {filtered_data.get('timestamp', '')[:19]}")

    if not posts:
        print("❌ No posts found. Run Step 1c first.")
        exit(1)

    # ── Neo4j credentials ─────────────────────────────────────
    print(f"\n🔐 Neo4j connection")
    print(f"   ℹ️  Find all 3 values in console.neo4j.io → instance → Connect tab")
    neo4j_uri  = input(f"   URI (neo4j+s://xxxxxxxx.databases.neo4j.io): ").strip()
    neo4j_user = input(f"   Username [neo4j]: ").strip() or "neo4j"
    neo4j_pwd  = getpass("   Password: ")

    # ── Verify connection before ingesting ───────────────────
    print(f"\n🔌 Testing connection as '{neo4j_user}'...")
    agent = None
    try:
        agent = Neo4jAgent(
            uri      = neo4j_uri,
            user     = neo4j_user,
            password = neo4j_pwd,
            run_id   = PIPELINE_RUN_ID,
        )
        with agent.driver.session() as session:
            result = session.run("RETURN 1 AS ok")
            result.single()
        print(f"   ✓ Auth OK — connected as '{neo4j_user}'")
    except Exception as e:
        msg = str(e)
        print(f"\n❌ Connection failed: {msg}")
        print(f"\n   💡 Exact steps to fix:")
        print(f"      1. Open console.neo4j.io")
        print(f"      2. Click your instance name")
        print(f"      3. Click the 'Connect' tab (not Overview)")
        print(f"      4. You will see: URI, Username, Password")
        print(f"      5. Click 'Reset Database Password' if needed")
        print(f"      6. Copy all 3 exactly — username is often 'neo4j' but verify")
        print(f"      7. The password shown there is the DB password, not your login")
        if agent:
            agent.close()
        raise

    log = Neo4jLog(pipeline_run=PIPELINE_RUN_ID)   # pre-init so except block can always access it
    try:
        log = agent.ingest(posts)
        print_graph_summary(log, posts)
        print_cypher_queries(log)

        # Save log
        with open(OUTPUT_FILE, "w") as f:
            json.dump(asdict(log), f, indent=2, default=str)
        print(f"\n✅ Saved: {OUTPUT_FILE}")

    except Exception as e:
        print(f"\n❌ Ingest error: {e}")
        log.status = "error"
        log.error  = str(e)
        with open(OUTPUT_FILE, "w") as f:
            json.dump(asdict(log), f, indent=2, default=str)
        raise
    finally:
        agent.close()

    # ── Pipeline status ───────────────────────────────────────
    print(f"\n{'='*60}")
    print(f"Pipeline:")
    print(f"  ✅ Tavily Scout   → 226 posts")
    print(f"  ✅ Scout Filter   → {len(posts)} posts")
    print(f"  ✅ Neo4j Agent    → graph upserted ({log.posts_ingested} nodes)")
    print(f"  🧠 Reka Agent    ← next: feed scout_filtered.json + neo4j_upsert_log.json")
    print(f"{'='*60}")

📂 Loading scout_filtered.json...
   5 posts loaded
   Source run: 2026-02-28T01:02:19

🔐 Neo4j connection
   ℹ️  Find all 3 values in console.neo4j.io → instance → Connect tab
   URI (neo4j+s://xxxxxxxx.databases.neo4j.io): neo4j+s://da0fdf56.databases.neo4j.io
   Username [neo4j]: da0fdf56
   Password: ··········

🔌 Testing connection as 'da0fdf56'...
✅ Connected to Neo4j — run: run_20260228_011437
   ✓ Auth OK — connected as 'da0fdf56'

🕸️  NEO4J AGENT — ingesting 5 posts

⚙️  Setting up constraints...
   ✓ CREATE CONSTRAINT topic_name IF NOT EXISTS FOR (t:Topic) REQ...
   ✓ CREATE CONSTRAINT platform_name IF NOT EXISTS FOR (p:Platfor...
   ✓ CREATE CONSTRAINT post_url IF NOT EXISTS FOR (p:Post) REQUIR...
   ✓ CREATE CONSTRAINT signal_word IF NOT EXISTS FOR (s:Signal) R...
   ✓ CREATE CONSTRAINT trend_id IF NOT EXISTS FOR (t:Trend) REQUI...

📥 Upserting nodes + relationships...
   [01/5] youtube    | Technologies              | Google IO 2025 Recap in 17 Minutes - YouTube
   [02/5] y